In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [54]:
folder = "../dataset/"
#basic_stats
names=['zipcode','date','category','merchant','card', 'payment', 'avg', 'max', 'min', 'std']
basic_stats = pd.io.parsers.read_table(folder + 'basic_stats000',sep='\t', names=names, parse_dates=["date"])

#customer_zipcodes
names=['zipcode','date','category','zipcodecustomer','merchant','card', 'payment', 'avg', 'max', 'min', 'std']
customer_zipcodes = pd.io.parsers.read_table(folder + 'customer_zipcodes000',sep='\t', names=names, parse_dates=["date"])

#age distribution
names=['zipcode','date','category','ageinterval','merchant','card', 'payment', 'avg', 'max', 'min', 'std']
age_distribution = pd.io.parsers.read_table(folder + 'age_distribution000',sep='\t', names=names, parse_dates=["date"])

#gender distribution
names=['zipcode','date','category','gender','merchant','card', 'payment', 'avg', 'max', 'min', 'std']
gender_distribution = pd.io.parsers.read_table(folder + 'gender_distribution000',sep='\t', names=names, parse_dates=["date"])

#payment_distribution 
names=['zipcode','date','category','paymentinterval','merchant','card', 'payment', 'avg', 'max', 'min', 'std']
payment_distribution = pd.io.parsers.read_table(folder + 'payment_distribution000',sep='\t', names=names, parse_dates=["date"])

#demographic distribution
names=['zipcode','date','category','ageinterval','gender','merchant','card', 'payment', 'avg', 'max', 'min', 'std']
demographic_distribution = pd.io.parsers.read_table(folder + 'demographic_distribution000',sep='\t', names=names, parse_dates=["date"])

#expenditure-time_curve
names=['zipcode','date','day_of_week','hour','merchant','card', 'payment', 'avg', 'max', 'min', 'std']
expenditure_time_curve = pd.io.parsers.read_table(folder + 'expenditure-time_curve000',sep='\t', names=names, parse_dates=["date"])


basic_stats["amount"] = basic_stats["payment"]* basic_stats["avg"]
basic_stats["avgpaybymerch"] = basic_stats["payment"]/ basic_stats["merchant"]
basic_stats["amountbymerch"] = basic_stats["amount"]/ basic_stats["merchant"]
basic_stats["avgReppays"] = basic_stats["payment"] / basic_stats["card"]
basic_stats["weekday"] = basic_stats["date"].map(lambda d: (d.weekday()))
basic_stats["day"] = basic_stats["date"].map(lambda d: ('{0:%d}-{0:%a}'.format(d)))

customer_zipcodes["amount"] = customer_zipcodes["payment"]* customer_zipcodes["avg"]
customer_zipcodes["avgpaybymerch"] = customer_zipcodes["payment"]/ customer_zipcodes["merchant"]
customer_zipcodes["amountbymerch"] = customer_zipcodes["amount"]/ customer_zipcodes["merchant"]
customer_zipcodes["avgReppays"] = customer_zipcodes["payment"] / customer_zipcodes["card"]
customer_zipcodes["weekday"] = customer_zipcodes["date"].map(lambda d: (d.weekday()))
customer_zipcodes["day"] = customer_zipcodes["date"].map(lambda d: ('{0:%d}-{0:%a}'.format(d)))

age_distribution["amount"] = age_distribution["payment"]* age_distribution["avg"]
gender_distribution["amount"] = gender_distribution["payment"]* payment_distribution["avg"]
payment_distribution["amount"] = payment_distribution["payment"]* payment_distribution["avg"]
demographic_distribution["amount"] = demographic_distribution["payment"]* demographic_distribution["avg"]
expenditure_time_curve["amount"] = expenditure_time_curve["payment"]* expenditure_time_curve["avg"]


In [44]:
#query from app will have:
#  time/date -> hour/day of week
#  category
#  Age
#  gender
#  amount ?? depends on number of persons (how many will you be?)
#  location -> x,y 
#  weather

In [83]:
tables =['basic_stats' ,'customer_zipcodes','age_distribution','gender_distribution','payment_distribution','demographic_distribution','expenditure_time_curve']

Where="where zipcode=8172 and date= '2014-07-01 00:00:00'"
for table in tables:
    query="""
    SELECT sum(amount), min(min), max(max), sum(payment)
    FROM """ + table +""" """ + Where
    dfquery= pysqldf(query)
    print table 
    print dfquery



basic_stats
   sum(amount)  min(min)  max(max)  sum(payment)
0     78293.54      0.89   1434.87          1632
customer_zipcodes
   sum(amount)  min(min)  max(max)  sum(payment)
0     65891.79      1.05   1434.87          1359
age_distribution
   sum(amount)  min(min)  max(max)  sum(payment)
0     55527.67      1.05   1434.87          1136
gender_distribution
   sum(amount)  min(min)  max(max)  sum(payment)
0     62947.99      0.89   1434.87          1598
payment_distribution
   sum(amount)  min(min)  max(max)  sum(payment)
0     48976.32      0.89       298          1382
demographic_distribution
   sum(amount)  min(min)  max(max)  sum(payment)
0     53789.11      1.05   1434.87          1068
expenditure_time_curve
   sum(amount)  min(min)  max(max)  sum(payment)
0     84076.19       0.4   1434.87          1769


In [77]:
query="""
SELECT *
FROM basic_stats   
WHERE max=25000""" 
dfquery= pysqldf(query)
print dfquery

#desanonimización: alguien el dia 7-7-2014 a las 20 se gasto 25.000 Eur en ropa en Rebajas :D


   zipcode                 date    category  merchant  card  payment      avg  \
0     8012  2014-07-07 00:00:00  es_fashion        75   432      449   154.38   
1     8021  2014-07-18 00:00:00   es_travel         5    13       20  1857.01   

     max   min          std    amount  avgpaybymerch  amountbymerch  \
0  25000  1.98  1416.559965  69316.62       5.986667       924.2216   
1  25000  8.60  5459.402917  37140.20       4.000000      7428.0400   

   avgReppays  weekday     day  
0    1.039352        0  07-Mon  
1    1.538462        4  18-Fri  


In [73]:
tables =['basic_stats' ,'customer_zipcodes','age_distribution','gender_distribution','payment_distribution','demographic_distribution','expenditure_time_curve']

Where="where zipcode=8172 and date= '2014-07-01 00:00:00'"
for table in tables:
    query="""
    SELECT * FROM """+ table +"""
    Where max=(SELECT max(max) maxmax
        FROM """ + table +""" """ + Where +""")"""
    dfquery= pysqldf(query)
    print table 
    print dfquery



basic_stats
   zipcode                 date category  merchant  card  payment     avg  \
0     8172  2014-07-01 00:00:00  es_auto        10    22       22  261.38   

       max    min         std   amount  avgpaybymerch  amountbymerch  \
0  1434.87  25.41  295.614781  5750.36            2.2        575.036   

   avgReppays  weekday     day  
0           1        1  01-Tue  
customer_zipcodes
   zipcode                 date category zipcodecustomer  merchant  card  \
0     8172  2014-07-01 00:00:00  es_auto         unknown         8    19   

   payment     avg      max    min         std   amount  avgpaybymerch  \
0       19  276.78  1434.87  30.86  309.531664  5258.82          2.375   

   amountbymerch  avgReppays  weekday     day  
0       657.3525           1        1  01-Tue  
age_distribution
   zipcode                 date category ageinterval  merchant  card  payment  \
0     8172  2014-07-01 00:00:00  es_auto     unknown         8    19       19   

      avg      max    min 

In [82]:
tables =['basic_stats','gender_distribution' ,'expenditure_time_curve']

Where="where zipcode=8172 and date= '2014-07-01 00:00:00'"
for table in tables:
    query="""
    SELECT *
    FROM """ + table +""" """ + Where
    dfquery= pysqldf(query)
    print table 
    print dfquery

                    
#hacer la media de amount merchans etc... para 0172 resto del mes i utilizarlo?                    

basic_stats
   zipcode                 date               category  merchant  card  \
0     8172  2014-07-01 00:00:00                es_auto        10    22   
1     8172  2014-07-01 00:00:00  es_barsandrestaurants        41   229   
2     8172  2014-07-01 00:00:00             es_fashion        54   417   
3     8172  2014-07-01 00:00:00                es_food        53   284   
4     8172  2014-07-01 00:00:00              es_health        16   131   
5     8172  2014-07-01 00:00:00                es_home        14   106   
6     8172  2014-07-01 00:00:00       es_otherservices        20   163   
7     8172  2014-07-01 00:00:00       es_sportsandtoys         8    16   
8     8172  2014-07-01 00:00:00                es_tech        13    30   
9     8172  2014-07-01 00:00:00   es_wellnessandbeauty        33   156   

   payment     avg      max    min         std    amount  avgpaybymerch  \
0       22  261.38  1434.87  25.41  295.614781   5750.36       2.200000   
1      232   24.12   12

In [ ]:

					es_bbva
					es_contents
					es_hotelservices
					es_hyper
					es_leisure
					es_propertyservices
					es_transportation
					es_travel
